In [112]:
import tensorflow.compat.v1 as tf
from tensorflow import feature_column
from tensorflow.keras import layers
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, Conv2D
from tensorflow.keras.optimizers import Adadelta, Adam, SGD

# from tensorflow import keras
# from tensorflow.keras.models import Sequential # 케라스의 Sequential()을 임포트
# from tensorflow.keras.layers import Dense # 케라스의 Dense()를 임포트
# from tensorflow.keras import optimizers # 케라스의 옵티마이저를 임포트
%matplotlib inline

In [113]:
seed = 0
# a = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))
# b = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))

# tensorflow, numpy 랜덤 값을 설정합니다.
tf.set_random_seed(seed)
np.random.seed(seed)

In [114]:
#doginfo.csv파일 데이터를 pandas를 이용해 읽어옵니다.
dog_data=pd.read_csv("doginfo2.csv")
# dog_train=pd.read_csv("testDog2.csv")
# kindCd=pd.read_csv("kindCd.csv")

dog_data = dog_data.dropna(axis=0)
# dog_train = dog_train.dropna(axis=0)
# kindCd_data = kindCd.dropna(axis=0)

dog_data
dog_train
# print(dog_train)
# print(kindCd)

,neuterYn,sexCd,weight,noticeDays,age2,processState
0,N,M,0.8,10,0,1
1,N,M,5.0,10,1,0
2,N,F,5.0,10,13,0
3,N,M,5.0,10,3,0
4,N,M,15.0,10,7,0
...,...,...,...,...,...,...
606,Y,M,6.4,11,10,0
607,U,F,1.7,11,0,0
608,U,F,1.7,11,0,1
609,U,F,3.6,11,2,1


In [115]:
# kindCd = np.array(kindCd_data, dtype = np.float64)
# kindCd_train=np.array(kindCd_data, dtype=np.float64)

# kindCd = kindCd.reshape(177)
# kindCd_train=kindCd_train.reshape(177)
# print(kindCd)
# print(kindCd.shape)

In [116]:
# # kindNum을 원핫 인코딩
# kindCd = pd.concat((pd.get_dummies(dog_data.kindNum, columns=kindCd), pd.DataFrame(columns=kindCd))).fillna(0)
# # kindCd

# # 학습데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
# dog_data.drop(['kindNum'], axis='columns', inplace=True)
# dog_data = pd.concat([dog_data, kindCd], axis=1)


# # kindNum을 원핫 인코딩
# kindCd_train = pd.concat((pd.get_dummies(dog_train.kindNum, columns=kindCd_train), pd.DataFrame(columns=kindCd_train))).fillna(0)

# # 테스트데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
# dog_train.drop(['kindNum'], axis='columns', inplace=True)
# dog_train = pd.concat([dog_train, kindCd_train], axis=1)
# dog_data
# # dog_train

In [117]:
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis = 1)
        data = data.drop(column, axis = 1)
    return data

In [118]:
dummy_columns = ["neuterYn", "sexCd"]
data = dummy_data(dog_data, dummy_columns)
# train_data = dummy_data(dog_train, dummy_columns)
data.head()
# train_data.head()
# print(type(dog_data))
# print(type(data))

print(data)

data = np.array(data, dtype = np.float64)
# train_data = np.array(train_data, dtype = np.float64)

# data
# print(data.shape)
# print(len(train_data))
# print(train_data.shape)
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), '훈련 샘플')
print(len(val), '검증 샘플')
print(len(test), '테스트 샘플')

       weight  noticeDays  age2  processState  neuterYn_N  neuterYn_U  \
0        7.46          10    12             0           1           0   
1        7.00          14     1             1           1           0   
2        4.50          11     2             0           0           1   
3       10.00           8     1             0           1           0   
4        6.00           8     4             0           1           0   
...       ...         ...   ...           ...         ...         ...   
22787    1.00          10     0             1           1           0   
22788    1.00          10     0             1           1           0   
22789    1.00          10     0             1           1           0   
22790    6.00          12     3             0           0           1   
22791    3.50          10     0             0           1           0   

       neuterYn_Y  sexCd_F  sexCd_M  sexCd_Q  
0               0        1        0        0  
1               0        0   

In [119]:
a = train[:, :3]
b = train[:, 4:]

# print(a[0,:])
# print(b)

#numpy 배열에서 데이터 변화요인(kindCd, neuterYn, sexCd, weight, noticeDays, age2)으로 사용할 데이터를 뽑아냅니다.
xData = np.concatenate([a, b], axis = 1)

a = test[:, :3]
b = test[:, 4:]
testX = np.concatenate([a, b], axis = 1)

a = val[:, :3]
b = val[:, 4:]
valX = np.concatenate([a, b], axis = 1)

print(xData)
type(xData)
xData.shape
testX.shape

[[18.  10.   4.  ...  1.   0.   0. ]
 [ 7.5 10.   2.  ...  1.   0.   0. ]
 [15.  10.   2.  ...  0.   1.   0. ]
 ...
 [ 4.8 10.   6.  ...  1.   0.   0. ]
 [ 2.  10.   0.  ...  0.   1.   0. ]
 [ 5.6 10.   2.  ...  1.   0.   0. ]]


(4559, 9)

In [120]:
#numpy배열에서 결과(입양여부)로 사용할 데이터를 뽑아냅니다.
yData=train[:,[3]]
testY = test[:,[3]]
valY = val[:,[3]]

# yData = yData.astype('int32')
print(yData)

[[1.]
 [1.]
 [1.]
 ...
 [0.]
 [1.]
 [1.]]


In [121]:
from sklearn import preprocessing

mm_scaler = preprocessing.MinMaxScaler()
xData = mm_scaler.fit_transform(xData)

In [ ]:
from keras import optimizers

model=Sequential()
# model.add(Dense(1, input_dim=9, activation='sigmoid'))


model.add(Dense(32, input_shape = (9, )))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9)

model.compile(optimizer='sgd' ,loss='binary_crossentropy', metrics=['accuracy', 'binary_crossentropy'])

hist = model.fit(xData,yData, batch_size=32, epochs=100, validation_data=(valX, valY), verbose=1)

loss_and_metric = model.evaluate(testX, testY, batch_size = 32, verbose = 0)
print("train, loss and metric : {}".format(loss_and_metric))

Train on 14586 samples, validate on 3647 samples
Epoch 1/100
14586/14586 [==============================] - 2s 162us/sample - loss: 0.7040 - acc: 0.5128 - binary_crossentropy: 0.7040 - val_loss: 0.8804 - val_acc: 0.4546 - val_binary_crossentropy: 0.8804
Epoch 2/100
14586/14586 [==============================] - 2s 118us/sample - loss: 0.6919 - acc: 0.5330 - binary_crossentropy: 0.6919 - val_loss: 0.8265 - val_acc: 0.4535 - val_binary_crossentropy: 0.8265
Epoch 3/100
14586/14586 [==============================] - 2s 117us/sample - loss: 0.6902 - acc: 0.5376 - binary_crossentropy: 0.6902 - val_loss: 0.7899 - val_acc: 0.4505 - val_binary_crossentropy: 0.7899
Epoch 4/100
14586/14586 [==============================] - 2s 153us/sample - loss: 0.6886 - acc: 0.5479 - binary_crossentropy: 0.6886 - val_loss: 0.7693 - val_acc: 0.4461 - val_binary_crossentropy: 0.7693
Epoch 5/100
14586/14586 [==============================] - 3s 181us/sample - loss: 0.6878 - acc: 0.5503 - binary_crossentropy: 0.68

14586/14586 [==============================] - 3s 204us/sample - loss: 0.6820 - acc: 0.5608 - binary_crossentropy: 0.6820 - val_loss: 0.7812 - val_acc: 0.5739 - val_binary_crossentropy: 0.7812
Epoch 41/100
14586/14586 [==============================] - 3s 179us/sample - loss: 0.6825 - acc: 0.5603 - binary_crossentropy: 0.6825 - val_loss: 0.7858 - val_acc: 0.5733 - val_binary_crossentropy: 0.7858
Epoch 42/100
14586/14586 [==============================] - 2s 136us/sample - loss: 0.6820 - acc: 0.5603 - binary_crossentropy: 0.6820 - val_loss: 0.7973 - val_acc: 0.5758 - val_binary_crossentropy: 0.7973
Epoch 43/100
14586/14586 [==============================] - 2s 119us/sample - loss: 0.6825 - acc: 0.5601 - binary_crossentropy: 0.6825 - val_loss: 0.8068 - val_acc: 0.5728 - val_binary_crossentropy: 0.8068
Epoch 44/100
14586/14586 [==============================] - 2s 116us/sample - loss: 0.6829 - acc: 0.5607 - binary_crossentropy: 0.6829 - val_loss: 0.8086 - val_acc: 0.5714 - val_binary_cros

In [144]:
# l2_model = keras.models.Sequential([
#     keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
#                        activation=tf.nn.relu, input_shape=(186,)),
#     keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
#                        activation=tf.nn.relu),
#     keras.layers.Dense(1, activation=tf.nn.sigmoid)
# ])


def mlp_model():
    model = Sequential()

#     model = Sequential([
#     Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
#                        activation=tf.nn.relu, input_shape=(186,)),
#     Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
#                        activation=tf.nn.relu),
#     Dense(1, activation=tf.nn.sigmoid)
# ])

    model.add(Dense(16, input_shape = (9, )))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dense(32))
    model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(Dense(32))
#     model.add(Activation('relu'))
#     model.add(Dense(50, input_shape = (186, )))
#     model.add(BatchNormalization())                    # Add Batchnorm layer before Activation
#     model.add(Activation('sigmoid'))    
#     model.add(Dense(50))
#     model.add(BatchNormalization())                    # Add Batchnorm layer before Activation
#     model.add(Activation('sigmoid'))    
#     model.add(Dense(50))
#     model.add(BatchNormalization())                    # Add Batchnorm layer before Activation
#     model.add(Activation('sigmoid'))    
#     model.add(Dense(50))
    model.add(BatchNormalization())                    # Add Batchnorm layer before Activation
    model.add(Dense(1))
    model.add(Activation('sigmoid'))   
#     model.add(Dropout(0.5))
#     model.add(Dense(1))
#     model.add(Activation('softmax'))

#     sgd=optimizers.SGD(lr=0.01, momentum=0.9)

    model.compile(optimizer=Adadelta(rho=0.95),
                     loss='binary_crossentropy',
                     metrics=['accuracy', 'binary_crossentropy'])
    return model

# mlp_model(l2_model)

# sgd=optimizers.SGD(lr=0.01)

# l2_model.compile(optimizer='sgd',
#                  loss='binary_crossentropy',
#                  metrics=['accuracy', 'binary_crossentropy'])

l2_model = mlp_model()

l2_model_history = l2_model.fit(xData, yData,
                                epochs=100,
                                shuffle = True,
                                batch_size=32,
                                validation_data=(testX, testY),
                                verbose=1)

l2_model.summary()

loss_and_metric = l2_model.evaluate(testX, testY, batch_size = 32, verbose = 0)
print("train, loss and metric : {}".format(loss_and_metric))


Train on 22792 samples, validate on 611 samples
Epoch 1/100
22792/22792 [==============================] - 1s 56us/sample - loss: 0.8675 - acc: 0.5316 - binary_crossentropy: 0.8675 - val_loss: 0.8734 - val_acc: 0.5254 - val_binary_crossentropy: 0.8734
Epoch 2/100
22792/22792 [==============================] - 1s 43us/sample - loss: 0.8615 - acc: 0.5330 - binary_crossentropy: 0.8615 - val_loss: 0.8591 - val_acc: 0.5303 - val_binary_crossentropy: 0.8591
Epoch 3/100
22792/22792 [==============================] - 1s 44us/sample - loss: 0.8526 - acc: 0.5377 - binary_crossentropy: 0.8526 - val_loss: 0.8585 - val_acc: 0.5286 - val_binary_crossentropy: 0.8585
Epoch 4/100
22792/22792 [==============================] - 1s 43us/sample - loss: 0.8478 - acc: 0.5365 - binary_crossentropy: 0.8478 - val_loss: 0.8535 - val_acc: 0.5368 - val_binary_crossentropy: 0.8535
Epoch 5/100
22792/22792 [==============================] - 1s 43us/sample - loss: 0.8399 - acc: 0.5392 - binary_crossentropy: 0.8399 - v

22792/22792 [==============================] - 1s 41us/sample - loss: 0.6818 - acc: 0.5893 - binary_crossentropy: 0.6818 - val_loss: 0.7064 - val_acc: 0.5630 - val_binary_crossentropy: 0.7064
Epoch 81/100
22792/22792 [==============================] - 1s 40us/sample - loss: 0.6813 - acc: 0.5914 - binary_crossentropy: 0.6813 - val_loss: 0.7039 - val_acc: 0.5630 - val_binary_crossentropy: 0.7039
Epoch 82/100
22792/22792 [==============================] - 1s 41us/sample - loss: 0.6796 - acc: 0.5890 - binary_crossentropy: 0.6796 - val_loss: 0.7063 - val_acc: 0.5679 - val_binary_crossentropy: 0.7063
Epoch 83/100
22792/22792 [==============================] - 1s 42us/sample - loss: 0.6826 - acc: 0.5899 - binary_crossentropy: 0.6826 - val_loss: 0.7046 - val_acc: 0.5614 - val_binary_crossentropy: 0.7046
Epoch 84/100
22792/22792 [==============================] - 1s 41us/sample - loss: 0.6821 - acc: 0.5858 - binary_crossentropy: 0.6821 - val_loss: 0.7033 - val_acc: 0.5630 - val_binary_crossentr

In [130]:
# for i in range(len(train_data)):
#     new_x=testX[i, :].reshape(1,186)
#     print('입양 확률 : %8.4f%%' % (l2_model.predict(new_x)*100))
l2_model.save('l2_model_yj')